# module import

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

## Data Partitioning

In [2]:
train = pd.read_csv("short_prac_train.csv", sep=',')
test = pd.read_csv("short_prac_test.csv", sep=',')

In [3]:
train_X = train.iloc[:, 1:785]/255
train_Y = train.iloc[:,0]
test_X = test.iloc[:, 1:785]/255
test_Y = test.iloc[:,0]

In [4]:
test_Y = pd.DataFrame(test_Y)
train_Y = pd.DataFrame(train_Y)

In [5]:
test_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 784 entries, pixel0 to pixel783
dtypes: float64(784)
memory usage: 6.0 MB


## Input 정의

In [6]:
x = tf.placeholder(tf.float32, [None, 784])

In [7]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

# 1st layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 2nd layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Dense
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32) # set 1.0 if inference
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### label과 momentum optimizer 정의

In [8]:
y = tf.placeholder(tf.int64, [None, 1])
y_ = tf.one_hot(indices = y, depth = 10)
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.01
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,10000, 1 - 1e-6, staircase=True)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y_conv))
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum = 0.9, use_nesterov=True).minimize(cost, global_step = global_step)

## minibatch

In [9]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

num_epochs = 30
num_data = train_X.shape[0]
batch_size = 50
for epoch in range(num_epochs):
    s_idx = np.random.permutation(num_data)
    
    for idx in range(0, num_data, batch_size):
        batch_x = train_X.iloc[s_idx[idx: idx + batch_size].tolist(),:]
        batch_y = train_Y.iloc[s_idx[idx: idx + batch_size].tolist()]
        sess.run(optimizer, feed_dict={x:batch_x, y:batch_y, keep_prob:0.5})
        
pred = sess.run(y_conv, feed_dict = {x: test_X, keep_prob:1.0})
pred_d = tf.argmax(pred,1)

print(confusion_matrix(test_Y, sess.run(pred_d)))
print(accuracy_score(test_Y, sess.run(pred_d)))

[[ 98   0   0   0   0   0   0   0   1   1]
 [  0  99   0   0   1   0   0   0   0   0]
 [  0   0  99   0   0   0   0   1   0   0]
 [  0   0   0 100   0   0   0   0   0   0]
 [  0   1   0   0  98   0   0   1   0   0]
 [  0   0   0   0   0 100   0   0   0   0]
 [  0   0   0   0   0   3  96   0   1   0]
 [  0   0   0   0   0   0   0  99   0   1]
 [  0   0   1   0   0   0   0   0  99   0]
 [  0   0   0   0   1   0   0   1   0  98]]
0.986


## accuracy_score=> 0.986